<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/U11/ANN11_LSTM_Shakespeare_SOLUTION.ipynb)

# Dichten wie Shakespeare ...

## Rekurrente Netze - LSTMs

- LSTMs führen das Konzept eines Zustands für jede Schicht in einem rekurrenten Netz ein. 
- Der Zustand fungiert als sein Speicher. Man kann sich das wie das Hinzufügen von Attributen zu einer Klasse in der objektorientierten Programmierung vorstellen. 
- Die Attribute des Speicherzustands werden mit jedem Trainingsbeispiel aktualisiert.

- In LSTMs sind die Regeln, welche die im State (Speicher) gespeicherten Informationen bestimmen, trainierte neuronale Netze - darin liegt der Magie (die Intelligenz?). Sie können trainiert werden, um zu lernen, was sie sich merken sollen, während gleichzeitig der Rest des rekurrenten Netzes lernt, das Ziel-Label vorherzusagen! 
- Mit der Einführung eines Speichers (memory) und eines Zustands (state) ist das Modell in der Lage, Abhängigkeiten zu lernen, die sich nicht nur über ein oder zwei Token, sondern über die die Gesamtheit jeder Datenprobe erstrecken. 

Mit diesen langfristigen Abhängigkeiten in der Hand, kann man über die Wörter selbst hinausgehen und etwas Tieferes über die Sprache herausfinden. Mit LSTMs stehen dem Modell Muster zur Verfügung, die der Mensch als selbstverständlich ansieht und auf einer unterbewussten Ebene verarbeitet. Und mit diesen Mustern können Sie nicht nur
Muster genauer vorhersagen, sondern auch **neue Texte generieren**. Der Stand der Technik in diesem Bereich ist noch lange nicht perfekt, aber die aber die Ergebnisse, die Sie sehen werden, selbst in Ihren Spielzeugbeispielen, sind beeindruckend.



## Modellierung von Sprache auf Zeichenebene

Worte haben eine Bedeutung - da sind wir uns alle einig. Die Modellierung natürlicher Sprache mit diesen
Grundbausteinen zu modellieren, erscheint daher nur natürlich (WordVectors). Die Verwendung dieser Modelle zur Beschreibung von Bedeutung, Gefühle, Absichten und alles andere in Form dieser atomaren Strukturen zu beschreiben, scheint
ebenfalls natürlich. 

Aber natürlich sind Wörter überhaupt nicht atomar. Wie Sie vorhin gesehen haben, bestehen sie aus kleineren Wörtern, Wortstämmen, Phonemen und so weiter. Aber sie sind auch, und das ist noch grundlegender, eine **Folge von Zeichen**.


- Bei der *Modellierung von Sprache* ist ein Grossteil der Bedeutung *auf der Zeichenebene* verborgen.
- *Intonationen* in der Stimme, *Alliterationen*, *Reime* - all das kann modelliert werden, wenn man wenn man die Dinge bis auf die Zeichenebene herunterbricht. 

Viele dieser Muster sind in einem Text enthalten, wenn man den Text daraufhin untersucht, welches Zeichen $x_k$ nach welchem folgt, unter Berücksichtigung (conditional probability) der Zeichen, die vorangegangen sind.

$$ p(x_k \vert x_{k-1}, x_{k-1}, \dots , x_{k-m})$$


In diesem Paradigma wird ein Leerzeichen, ein Komma oder ein Punkt einfach zu einem weiteren Zeichen.
Und da das LSTM-Netz die **Bedeutung von Sequenzen lernt**, ist es gezwungen, Muster auf niedrigerer Ebene zu finden.
Wenn das Modell nach einer bestimmten Anzahl von Silben ein sich wiederholendes Suffix erkennt, das sich
reimt, könnte dies ein Hinweis auf eine Bedeutung wie vielleicht Heiterkeit oder Spott sein.

- Mit einer ausreichend grossen Trainingsmenge beginnen sich diese Muster herauszukristallisieren. Und da es in der englischen Sprache viel weniger unterschiedliche Buchstaben als Wörter gibt, muss sich das Modell um eine eine geringere Vielfalt an Eingabevektoren kümmern (*Dimensionsreduktion* des Inputs).
- **Das Trainieren eines Modells auf Buchstabenebene ist jedoch schwierig**. Die Muster und langfristigen Abhängigkeiten, die auf der Zeichenebene zu finden sind, können von Text zu Text sehr unterschiedlich sein. Sie können diese Muster finden, aber sie lassen sich möglicherweise nicht so gut verallgemeinern. 


In [ ]:
import os
import pathlib
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics


### (a) Herunterladen und Speichern des Datensatzes 

1. **Herunterladen des Textes:**
   - Es wird eine Verbindung zu einer URL aufgebaut, wo sich ein Text mit Werken von Shakespeare befindet.
   - Der komplette Text wird aus dem Internet geladen und im Arbeitsspeicher gespeichert.

2. **Speichern auf dem Computer:**
   - Der geladene Text wird in eine neue Datei namens `shakespeare.txt` geschrieben.
   - Diese Datei liegt dann lokal auf deinem Rechner unter `/data/`.

In [ ]:
import requests

# URL of the text file
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/678ae4873e82f52ae1b563e32c12c6837fd5ae78/data/Gutenberg/shakespeare.txt"

# Download the file
response = requests.get(url)
shakespeare_text = response.text

# Save the text to a local file (optional)
with open("./data/shakespeare.txt", "w", encoding="utf-8") as file:
    file.write(shakespeare_text)

print("Text file downloaded and saved.")

In [ ]:
with open("./data/shakespeare.txt") as f:
    text = f.read()


In [ ]:
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))


### (b) Generieren Sie ein `dictionary`, das jedem Character einen Index zuweist und umgekehrt.

In [ ]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
print(chars)

In [ ]:
print(char_indices)

### (c) Eine Textstelle anzeigen


In [ ]:

print(text[:500])

### (d) PyTorch Dataset class: Aufteilen der Textsequenzen in Teilstücke

Generieren Sie mit Hilfe eines LLM (ChatGPT, Gemini, ...) ein PyToch-Dataset, das folgende Aufgaben erfüllt:
- es genieriert einen Datensatz für die Sprachmodellierung auf Zeichenebene bei Shakespeare-Texten.
- dazu erstellt es ein Vokabular der Zeichen und codiert the Zeichenketten in Indizes und
```python
        self.chars = sorted(list(set(text)))
        self.char2idx = {c: i for i, c in enumerate(self.chars)}
        self.idx2char = {i: c for i, c in enumerate(self.chars)}
        self.vocab_size = len(self.chars)
```
- Gibt Sequenzen von Zeichenindizes und Beschriftungen des nächsten Zeichens zurück.
- Teilt den Input-Korpus in Zeichenketten der maximalen Länge `maxlen=40` auf. 
- Verwendet einen Stride von 3 Zeichen, sodass eine semi-redundante Repräsentation der Input-Sequenz entsteht.
- Erstellt gleichzeitig eine Liste `next_chars`, welche das nächste, folgende Zeichen nach dieser Zeichenkette speichert. Dies sind die zugehörigen Labels.

In [ ]:
class CharDataset(Dataset):
    """
    Dataset for character-level language modeling on Shakespeare texts.
    Returns sequences of character indices and next-character labels.
    """
    def __init__(self, data_dir: str, maxlen: int = 40, step: int = 3):
        text = ''
        for path in pathlib.Path(data_dir).iterdir():
            if path.is_file():
                with open(path, 'r', encoding='utf-8') as f:
                    text += f.read().lower()
        self.chars = sorted(list(set(text)))
        self.char2idx = {c: i for i, c in enumerate(self.chars)}
        self.idx2char = {i: c for i, c in enumerate(self.chars)}
        self.vocab_size = len(self.chars)
        self.maxlen = maxlen
        self.step = step
        self.sequences = []
        self.next_chars = []
        for i in range(0, len(text) - maxlen, step):
            self.sequences.append(text[i: i + maxlen])
            self.next_chars.append(text[i + maxlen])

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        # Return sequence of indices
        x = torch.tensor([self.char2idx[ch] for ch in seq], dtype=torch.long)
        # Target index
        y = torch.tensor(self.char2idx[self.next_chars[idx]], dtype=torch.long)
        return x, y

### (e) Pytorch DataLoader

Das `ShakespeareDataModule` ist ein **PyTorch Lightning**-Mechanismus zum Laden, Vorverarbeiten und Batchen eines Shakespeare-Textkorpus für ein Zeichen-basiertes LSTM-Modell. 
Es:

* Liest alle Shakespeare-Dateien aus einem Verzeichnis
* Zerlegt den Text in Sequenzen fester Länge
* Teilt die Daten in **Training**- und **Validierungs**-Sets
* Stellt jeweils passende `DataLoader` bereit


**Parameter im Konstruktor**

| Parameter    | Typ     | Beschreibung                                                                                     |
| ------------ | ------- | ------------------------------------------------------------------------------------------------ |
| `data_dir`   | `str`   | Pfad zum Verzeichnis mit den Shakespeare-Textdateien                                             |
| `batch_size` | `int`   | Anzahl der Sequenzen pro Batch beim Training/Validierung                                         |
| `val_split`  | `float` | Anteil (z. B. `0.05` = 5 %) für das Validierungs-Set                                             |
| `maxlen`     | `int`   | Länge jeder Eingabesequenz (Anzahl Zeichen)                                                      |
| `step`       | `int`   | Schrittweite beim Verschieben des Fensters über den Text (z. B. `3` für semi-redundante Samples) |



**Hinweis:** Dieses Modul trennt die Daten sauber vom Modell-Code. So könnt ihr verschiedene Modelle (LSTM, Transformer, …) einsetzen, ohne das Preprocessing anzupassen.


In [ ]:

class ShakespeareDataModule(pl.LightningDataModule):
    def __init__(self,
                 data_dir: str = 'data',
                 batch_size: int = 128,
                 val_split: float = 0.05,
                 maxlen: int = 40,
                 step: int = 3):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.val_split = val_split
        self.maxlen = maxlen
        self.step = step

    def setup(self, stage=None):
        dataset = CharDataset(self.data_dir, self.maxlen, self.step)
        total = len(dataset)
        val_size = int(total * self.val_split)
        train_size = total - val_size
        self.train_dataset, self.val_dataset = random_split(dataset, [train_size, val_size])
        self.vocab_size = dataset.vocab_size

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

### (f) Definieren Sie ein einfaches LSTM-Modell mit PyTorch Lightning

Implementieren Sie mit Hilfe eines LLM (ChatGPT, Gemini, ...) ein LSTM-Modell für die Vorhersage des nächsten Zeichens.

- Zu Beginn werden die Eingabeindizes über eine **Embedding-Schicht** in *dichte Vektoren* überführt, was gegenüber grossen, spärlichen One‑Hot-Repräsentationen deutlich speichereffizienter ist und es dem Modell erlaubt, semantische Ähnlichkeiten zwischen Zeichen zu lernen. 
- Anschliessend verarbeitet ein **Long Short‑Term Memory (LSTM)** diese eingebetteten Sequenzen, um zeitliche Abhängigkeiten im Text zu erfassen und langfristige Kontextinformationen im verborgenen Zustand zu speichern.
- Der letzte verborgene (hidden) Zustand des LSTM dient als **komprimierte Repräsentation der gesamten Eingabesequenz**. 
- Eine **Dropout-Schicht** aktiviert sich danach, um Überanpassung zu verhindern, indem sie zufällig Teile des neuronalen Netzwerks deaktiviert. 
- Ein **dichtes Zwischennetzwerk** mit nichtlinearer Aktivierung (ReLU) sorgt für zusätzliche Modellkapazität und stellt sicher, dass komplexe Zusammenhänge zwischen den Merkmalen gelernt werden können. 
- Ein erneutes **Dropout** verbessert die Generalisierungsfähigkeit weiter. 
- Abschliessend projiziert eine finale vollverknüpfte Schicht dieses Merkmal auf so viele Ausgabeneinheiten, wie Zeichen im Vokabular vorhanden sind, um für jedes mögliche Folgezeichen einen Rohwert (Logit) zu erzeugen.

Verwenden Sie folgende Parameter:
```python
class LitCharRNN(pl.LightningModule):
    def __init__(self,
                vocab_size: int,
                embed_dim: int = 64,
                hidden_size: int = 128,
                dropout: float = 0.2,
                lr: float = 5e-3):
```

Für die **Optimierung** kommt die Kreuzentropie als Verlustfunktion zum Einsatz. Sie kombiniert implizit Softmax und Negative Log-Likelihood und ist der Standard bei Mehrklassen-Klassifikationsaufgaben; die fehlende Notwendigkeit einer manuellen Softmax-Operation im Netzwerkaufbau erhöht die numerische Stabilität und hält den Trainingscode schlank. RMSprop dient als Optimierer, da er durch adaptive Lernraten und eine gewichtete Mittelung vergangener Gradienten gut mit den dynamischen Gradienten von rekursiven Netzen zurechtkommt.

Während des Trainings und der Validierung wird die Modellgenauigkeit mittels einer **Multiclass-Accuracy-Metrik** verfolgt. Dieser Ansatz ermöglicht eine präzise Überwachung, wie oft das Modell das nächste Zeichen korrekt vorhersagt, und liefert direkt interpretierbare Kennzahlen zur Modellgüte. Insgesamt bildet die Kombination aus Embedding, LSTM, Dropout, nichtlinearer Zwischenschicht und Kreuzentropie-Loss eine robuste Architektur für Zeichen-basiertes Sprachmodellieren, die sowohl Effizienz als auch Leistungsfähigkeit gewährleistet.


In [ ]:
class LitCharRNN(pl.LightningModule):
    def __init__(self,
                 vocab_size: int,
                 embed_dim: int = 64,
                 hidden_size: int = 128,
                 dropout: float = 0.2,
                 lr: float = 5e-3):
        super().__init__()
        self.save_hyperparameters()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(input_size=embed_dim,
                            hidden_size=hidden_size,
                            batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, vocab_size)
        self.train_accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=vocab_size)
        self.val_accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=vocab_size)

    def forward(self, x):
        # x: (batch, seq_len) of indices
        emb = self.embedding(x)  # (batch, seq_len, embed_dim)
        output, (h_n, c_n) = self.lstm(emb)
        last_hidden = h_n[-1]
        x = self.dropout(last_hidden)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        logits = self.fc2(x)
        return logits

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = self.train_accuracy(preds, y)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = self.val_accuracy(preds, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters(), lr=self.hparams.lr)

### (g) Softmax-Sampling

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Reproducible logits
np.random.seed(42)
logits = np.random.randn(10)

# Convert logits to base probabilities
base_probs = np.exp(logits) / np.sum(np.exp(logits))

# Softmax-with-temperature function (returns distribution)
def tempered_softmax(preds: np.ndarray, temperature: float = 1.0) -> np.ndarray:
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    return exp_preds / np.sum(exp_preds)

# Temperatures to visualize
temperatures = [0.1, 2, 5]
# Create a single figure with 3 subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, T in zip(axes, temperatures):
    dist = tempered_softmax(base_probs, T)
    ax.bar(range(10), dist)
    ax.set_title(f"T = {T}")
    ax.set_xlabel("Index")
    ax.set_ylabel("Probability")
    ax.set_ylim(0, dist.max() + 0.1)
    ax.grid(True)
    ax.set_xticks(range(10))
plt.tight_layout()
plt.show()



- Now, we generate out of a standard softmax-output a tempered_softmax output and sample from it. 
- The temperature parameter T controls the sharpness of the distribution. 
- A low T (e.g., 0.1) makes the distribution sharper, while a high T (e.g., 5) makes it flatter. 
- This allows us to control the randomness of the sampling process.
 

In [ ]:
def sample(preds: np.ndarray, temperature: float = 1.0) -> int:
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



### (h) Modell trainiren

In [ ]:
pl.seed_everything(42)

data_module = ShakespeareDataModule(data_dir='data', batch_size=128)
data_module.setup()

model = LitCharRNN(vocab_size=data_module.vocab_size)
logger = TensorBoardLogger('tb_logs', name='shakes_rnn')
early_stop = EarlyStopping(monitor='train_loss', patience=5, mode='min')
trainer = pl.Trainer(max_epochs=20, logger=logger, callbacks=[early_stop])
trainer.fit(model, datamodule=data_module)
torch.save(model.state_dict(), 'shakes_lstm_weights.pth')

### (i) Neue Texte generieren:

Sie mit der Temperatur oder den Startsequenzen.

In [ ]:
# Generate text
text_data = ''
for path in pathlib.Path('data').iterdir():
    if path.is_file():
        text_data += open(path, 'r').read().lower()
start_index = random.randint(0, len(text_data) - data_module.maxlen - 1)
seed = text_data[start_index: start_index + data_module.maxlen]
for diversity in [0.2, 0.5, 1.0, 1.2]:
    print(f"\n----- temperature: {diversity}")
    print(f"----- Generating with seed: '{seed}'")
    sentence = seed
    generated = seed
    for _ in range(400):
        x_pred = torch.tensor([[data_module.train_dataset.dataset.char2idx[ch] for ch in sentence]], dtype=torch.long).to(model.device)
        with torch.no_grad():
            preds = model(x_pred)                         # preds is a tensor
            tensor_probs = F.softmax(preds, dim=-1)       # softmax on tensor
            probs = tensor_probs.cpu().numpy().squeeze()  # then to NumPy
            #probs = F.softmax(logits, dim=-1).cpu().numpy().squeeze()
        next_idx = sample(probs, diversity)
        next_char = data_module.train_dataset.dataset.idx2char[next_idx]
        generated += next_char
        sentence = sentence[1:] + next_char
    print(generated)